# This program optimizes a stock portfolio

In [ ]:
!pip install yfinance

In [2]:
import pandas as pd
import numpy as np
import requests

In [3]:
def get_sp500():
    """
    Get list of S&P 500 stocks from wikipedia
    """
    sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    sp500_tickers = sp500_tickers[0]

    tickers = sp500_tickers['Symbol'].values.tolist()
    return tickers

companies = get_sp500()

In [7]:
import os
from pandas_datareader import data as pdr
import pandas as pd
#import fix_yahoo_finance as yf
import yfinance as yf

yf.pdr_override()

START_DATE = "2021-01-01"
END_DATE = "2021-04-01"

def build_stock_dataset(start=START_DATE, end=END_DATE):
    """
    Creates the dataset containing all stock prices
    :returns: stock_prices.csv
    """
    ticker_list = get_sp500()

    # Get all Adjusted Close prices for all the tickers in our list,
    # between START_DATE and END_DATE
    all_data = pdr.get_data_yahoo(ticker_list, start, end)
    stock_data = all_data["Adj Close"]

    # Remove any columns that hold no data, and print their tickers.
    stock_data.dropna(how="all", axis=1, inplace=True)
    missing_tickers = [
        ticker for ticker in ticker_list if ticker.upper() not in stock_data.columns
    ]
    print(f"{len(missing_tickers)} tickers are missing: \n {missing_tickers} ")
    # If there are only some missing datapoints, forward fill.
    stock_data.ffill(inplace=True)
    stock_data.to_csv("stock_prices.csv")
    
build_stock_dataset()

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
2 tickers are missing: 
 ['BRK.B', 'BF.B'] 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [10]:
# Load data
df = pd.read_csv("stock_prices.csv")
# Set date as index
df = df.set_index(pd.DatetimeIndex(df['Date'].values))
# remove date column
df.drop(columns=['Date'], axis=1, inplace=True)

In [11]:
df

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,...,V,VFC,VIAC,VLO,VMC,VNO,VRSK,VRSN,VRTX,VTR,VTRS,VZ,WAB,WAT,WBA,WDC,WEC,WELL,WFC,WHR,WLTW,WM,WMB,WMT,WRB,WRK,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-04,118.459747,15.130000,157.123825,128.997803,102.939789,95.746490,316.730011,108.274658,254.798035,485.339996,145.397141,49.523808,168.096970,296.839996,75.565613,80.116394,22.962521,42.886234,36.884716,131.399780,119.766922,105.239998,152.239166,526.460022,49.240002,107.287674,114.131126,156.809998,86.705444,11.262912,92.300003,118.185791,223.416199,186.397797,218.467239,3186.629883,283.160004,357.670013,317.519897,203.659470,...,217.107101,84.608238,36.498192,54.199249,146.207825,35.768425,202.540329,213.580002,229.429993,46.494473,18.540001,57.606892,72.437752,250.149994,41.009056,52.230000,89.027275,60.550266,29.541864,175.757339,203.083511,114.287422,19.762255,145.349930,64.835655,42.255943,285.254730,21.566744,32.643650,106.900002,65.173874,142.429993,40.221066,53.660934,99.407997,105.315231,152.927170,378.130005,42.397789,163.093948
2021-01-05,119.428276,15.430000,156.954056,130.592697,104.004242,96.996651,322.600006,109.614319,256.248596,485.690002,147.996002,50.487076,167.629364,301.700012,75.198364,80.008316,23.733208,42.955742,37.500618,131.439560,118.970070,105.419998,162.512772,543.650024,49.860001,106.443909,114.350456,157.360001,89.430267,11.342089,92.769997,119.004440,224.500443,185.307800,219.571518,3218.510010,286.010010,359.829987,314.051483,203.410538,...,213.866837,84.816978,36.887108,55.745575,147.245544,34.773232,201.631851,206.500000,228.580002,47.049866,18.100000,57.342598,73.345085,255.429993,40.771320,53.060001,88.741615,60.904129,30.367445,177.575531,201.388657,114.665619,20.362003,144.576218,64.276543,43.311096,290.698669,21.893663,32.952076,110.190002,64.538605,144.229996,42.159431,55.198383,99.288330,105.325180,155.573135,380.570007,43.069359,164.230499
2021-01-06,122.703293,15.520000,166.021576,126.196747,103.105804,105.340981,321.609985,109.386086,259.050293,466.309998,148.663147,52.532776,167.042374,302.869995,75.823692,80.646973,24.681744,44.613998,39.944366,137.208389,121.450272,101.529999,174.651627,540.390015,51.520000,109.183670,116.952499,157.759995,90.657944,11.658796,90.330002,124.046158,229.911880,195.940140,213.781525,3138.379883,285.869995,349.899994,327.227509,205.879745,...,211.982513,86.954048,40.088181,56.990417,160.406525,35.875751,199.725037,199.509995,226.649994,47.972218,18.490000,58.037601,78.918762,262.470001,42.623661,53.410000,89.864555,63.479488,32.515942,177.466232,203.083511,117.203575,21.138729,145.478897,66.482994,46.038570,293.815247,22.904140,33.121216,110.849998,65.967941,141.220001,43.235222,57.863960,103.476448,105.902420,160.006393,394.820007,47.908611,166.643158
2021-01-07,125.968323,15.380000,167.439621,130.502991,104.209320,109.270058,323.559998,110.447891,261.494385,477.739990,154.946228,52.066040,167.987534,315.200012,73.689629,78.495201,24.434729,44.365757,40.619877,138.590912,121.848701,102.809998,183.528824,558.359985,51.320000,109.263084,120.033089,155.750000,94.380875,11.559825,95.160004,124.195915,230.670868,198.308380,216.378067,3162.159912,295.970001,365.019989,336.257355,205.660706,...,213.168945,87.749237,40.008400,58.031025,161.653778,34.958614,200.633514,200.000000,232.899994,47.049866,18.520000,57.907001,81.700607,260.790009,44.832603,54.740002,86.939011,62.221294,33.252003,181.291367,204.628830,117.492203,21.178059,145.468964,65.903931,45.620491,303.724274,22.646568

In [12]:
assets = df.columns

In [14]:
!pip install PyPortfolioOpt

     |████████████████████████████████| 61kB 2.2MB/s 
     |████████████████████████████████| 1.3MB 5.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for cvxpy: filename=cvxpy-1.1.12-cp37-cp37m-linux_x86_64.whl size=2731667 sha256=82bb4edca631991a621218772742828b4d05dec1382ad584e1963df38507cefb
  Stored in directory: /root/.cache/pip/wheels/9b/62/55/1da181c05c710c5d99bd560edebec3bd6a61cb69acef9dc00e
Successfully built cvxpy
  Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [16]:
# Optimize the portfolio
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns

In [20]:
# Calculate expected anuallized returns

mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

/usr/local/lib/python3.7/dist-packages/pypfopt/risk_models.py:69: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."
/usr/local/lib/python3.7/dist-packages/pypfopt/risk_models.py:88: UserWarning: Could not fix matrix. Please try a different risk model.
  "Could not fix matrix. Please try a different risk model.", UserWarning


In [24]:
# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('A', 0.0), ('AAL', 0.08165), ('AAP', 0.0), ('AAPL', 0.0), ('ABBV', 0.0), ('ABC', 0.0), ('ABMD', 0.0), ('ABT', 0.0141), ('ACN', 0.0), ('ADBE', 0.0), ('ADI', 0.0), ('ADM', 0.0), ('ADP', 0.0), ('ADSK', 0.0), ('AEE', 0.02477), ('AEP', 0.0), ('AES', 0.0), ('AFL', 0.0), ('AIG', 0.0), ('AIZ', 0.0), ('AJG', 0.0), ('AKAM', 0.0), ('ALB', 0.0), ('ALGN', 0.0), ('ALK', 0.0), ('ALL', 0.0), ('ALLE', 0.0), ('ALXN', 0.0), ('AMAT', 0.05891), ('AMCR', 0.0), ('AMD', 0.0), ('AME', 0.0), ('AMGN', 0.0), ('AMP', 0.0), ('AMT', 0.0), ('AMZN', 0.0), ('ANET', 0.0), ('ANSS', 0.0), ('ANTM', 0.0), ('AON', 0.0), ('AOS', 0.0), ('APA', 0.0), ('APD', 0.0), ('APH', 0.0), ('APTV', 0.0), ('ARE', 0.0), ('ATO', 0.0), ('ATVI', 0.0), ('AVB', 0.0), ('AVGO', 0.0), ('AVY', 0.0), ('AWK', 0.0), ('AXP', 0.0), ('AZO', 0.0), ('BA', 0.0), ('BAC', 0.0), ('BAX', 0.0), ('BBY', 0.0), ('BDX', 0.0), ('BEN', 0.0), ('BIIB', 0.0), ('BIO', 0.0), ('BK', 0.0), ('BKNG', 0.0), ('BKR', 0.0), ('BLK', 0.0), ('BLL', 0.0), ('BMY', 0.0), ('B

(2.375066079764527, 0.10985521722442311, 21.437908360359117)

In [25]:
!pip install pulp

     |████████████████████████████████| 40.6MB 105kB/s 


In [27]:
# Get discrete allocation of each share per stock

from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

#How much money I want to put in USD
portfolio_val = 5000

latest_prices=get_latest_prices(df)
weights = cleaned_weights
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value= portfolio_val)
allocation, leftover = da.lp_portfolio()
print('Discrete Allocation ', allocation)
print('Funds Aemaining: $', leftover)

Discrete Allocation  {'AAL': 17, 'ABT': 1, 'AEE': 1, 'AMAT': 2, 'CCI': 1, 'DISCA': 5, 'DLR': 3, 'ED': 1, 'EXR': 5, 'FE': 8, 'HIG': 2, 'IRM': 10, 'KHC': 3, 'LB': 6, 'LH': 2, 'NUE': 2, 'PXD': 1, 'SJM': 2, 'STX': 1, 'WBA': 1, 'WEC': 1}
Funds Aemaining: $ 1.9444828033447266


In [32]:
# Create a function to get companies name
import requests
def get_company_name(symbol):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(symbol)

    result = requests.get(url).json()
    
    for x in result['ResultSet']['Result']:
        if x['symbol'] == symbol:
            return x['name']

company = get_company_name("MSFT")

print(company)

Microsoft Corporation


In [33]:
# Store company name in a list
company_name = []
for symbol in allocation:
  company_name.append(get_company_name(symbol))

In [34]:
# Get the discrete allocation values
discrete_allocation_list = []
for symbol in allocation:
  discrete_allocation_list.append(allocation.get(symbol))

In [37]:
# Create a df for the portfolio

portfolio_df = pd.DataFrame(columns=["Company_name", "Company_ticker", "Discrete_val"+str(portfolio_val)])
portfolio_df['Company_name']= company_name
portfolio_df['Company_ticker']= allocation
portfolio_df["Discrete_val"+str(portfolio_val)]= discrete_allocation_list

In [38]:
portfolio_df

,Company_name,Company_ticker,Discrete_val5000
0,American Airlines Group Inc.,AAL,17
1,Abbott Laboratories,ABT,1
2,Ameren Corporation,AEE,1
3,"Applied Materials, Inc.",AMAT,2
4,Crown Castle International Corp. (REIT),CCI,1
5,"Discovery, Inc.",DISCA,5
6,"Digital Realty Trust, Inc.",DLR,3
7,"Consolidated Edison, Inc.",ED,1
8,Extra Space Storage Inc.,EXR,5
9,FirstEnergy Corp.,FE,8
